Recalcuating Criticality Score Based on Statistical Analysis \\
Author: Blaise Swartwood

In [11]:
#importing required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import math

In [26]:
#accessing data file
df = pd.read_csv('CSDataAsCSV.csv', usecols=[i for i in range(6, 18) if i != 16 and i != 10 and i != 13 and i != 15])
df.head()

,created_since,updated_since,contributor_count,org_count,recent_release_count,updated_issues_count,issue_comment_frequency,criticality_score
0,131,0,5000,10,69,552,3.18,0.92520
1,209,0,2044,5,49,896,13.15,0.89559
2,162,0,3472,9,67,36122,2.00,0.88907
3,97,0,4367,5,65,86352,2.00,0.88400
4,151,0,3579,13,69,27449,2.00,0.87876


In [27]:
#creating a dataframe to hold the weights and maxthreshold values for calculation
data = [[0.5, 120], [2, 120], [2,5000], [2, 10],[0.5, 26],[2, 5000],[2, 15]]
df2 = pd.DataFrame(data, columns=['Weight','Max'])

#an updated change to remove the negative weight of updated_since
df['updated_since'] = 120 - np.minimum(df['updated_since'], 120)

In [28]:
df.head()

,created_since,updated_since,contributor_count,org_count,recent_release_count,updated_issues_count,issue_comment_frequency,criticality_score
0,131,120,5000,10,69,552,3.18,0.92520
1,209,120,2044,5,49,896,13.15,0.89559
2,162,120,3472,9,67,36122,2.00,0.88907
3,97,120,4367,5,65,86352,2.00,0.88400
4,151,120,3579,13,69,27449,2.00,0.87876


In [29]:
#getting the total weight and preparing the new criticality score column
weight = df2['Weight'].sum()
df['new_crit'] = 0

In [21]:
#implementing the criticality score algorithm using only these specific signals

for i in range(df.shape[0]):
    x = 0
    for index in range(7):
        a = df2['Weight'][index]
        t = df2['Max'][index]
        s = df.iloc[i, index]
        x += a*(np.log(1+s))/np.log(1+np.maximum(s,t))
    df.loc[i, 'new_crit'] = x/weight

In [23]:
#adding a new column to compare the % change from the original criticality score
df['percent_change'] = 100 * (df['new_crit']-df['criticality_score'])/df['criticality_score']

In [24]:
df.head(10)

,created_since,updated_since,contributor_count,org_count,recent_release_count,updated_issues_count,issue_comment_frequency,criticality_score,new_crit,percent_change
0,131,120,5000,10,69,552,3.18,0.92520,0.864971,-6.509810
1,209,120,2044,5,49,896,13.15,0.89559,0.890213,-0.600419
2,162,120,3472,9,67,36122,2.00,0.88907,0.875215,-1.558331
3,97,120,4367,5,65,86352,2.00,0.88400,0.839379,-5.047647
4,151,120,3579,13,69,27449,2.00,0.87876,0.883090,0.492728
5,282,120,1340,8,51,7121,3.71,0.87804,0.876493,-0.176175
6,234,120,5000,7,5,161955,2.00,0.87599,0.845336,-3.499398
7,213,120,5000,7,35,21279,2.00,0.87284,0.866079,-0.774585
8,69,120,5000,10,125,212,7.45,0.87088,0.885575,1.687323
9,126,120,3315,5,6,79245,2.00,0.86670,0.816877,-5.748533


In [25]:
#downloading results as a csv
df['repo'] = pd.read_csv('CSDataAsCSV.csv', usecols=[0])df.to_csv("newCriticalityScore.csv", index=False)